### Production Features Pipeline

This notebook is run daily from a Github Action. 

1. It scrapes the results from the previous day's games, performs feature engineering, and saves the results back to the Feature Store at Hopsworks.ai

2. It scrapes the upcoming games for today, and saves the blank records back into the Feature Store at Hopsworks.ai so that they can be accessed by the model for the prediction service via the streamlit app.

**Note:**
There are two options for webscraping in this notebook. 
Set the 'WEBSCRAPER' variable to either 'SCRAPINGANT' or 'SELENIUM' to choose which version to run.

1. SCRAPINGANT: Uses a webscraping service with a Python API, ScrapingAnt, which handles all the proxy server issues, but does require an account. The free account allows for 1000 page requests, which is more than enough for this project. Proxies are required when running this notebook from a Github Action or otherwise key data will fail to be scraped from NBA.com. 

2. SELENIUM: This option does not currently integrate proxy servers into the webscraping process, which can cause issues when scraping from certain locations, in particular Github Actions. 

In [1]:
# select web scraper; 'SCRAPINGANT' or 'SELENIUM'
# SCRAPINGANT requires a subscription but includes a proxy server

WEBSCRAPER = 'SCRAPINGANT'
#WEBSCRAPER = 'SELENIUM'

In [2]:
import os

import pandas as pd
import numpy as np

import hopsworks

from datetime import datetime, timedelta
from pytz import timezone

from src.webscraping import (
    get_new_games,
    activate_web_driver,
    get_todays_matchups,
)

from src.data_processing import (
    process_games,
    add_TARGET,
)

from src.feature_engineering import (
    process_features,
)

from src.hopsworks_utils import (
    save_feature_names,
    convert_feature_names,
)

import json

import time


from pathlib import Path  #for Windows/Linux compatibility
DATAPATH = Path(r'data')

**Load API keys**

In [3]:
from dotenv import load_dotenv

load_dotenv()

try:
    HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
except:
    raise Exception('Set environment variable HOPSWORKS_API_KEY')

# if scrapingant is chosen then set the api key, otherwise load the selenium webdriver
if WEBSCRAPER == 'SCRAPINGANT':
    try:
        SCRAPINGANT_API_KEY = os.environ['SCRAPINGANT_API_KEY']
    except:
        raise Exception('Set environment variable SCRAPINGANT_API_KEY')
    driver = None
    
elif WEBSCRAPER == 'SELENIUM':
    driver = activate_web_driver('chromium')
    SCRAPINGANT_API_KEY = ""
    



**Scrape New Completed Games and Format Them**

In [4]:


df_new = get_new_games(SCRAPINGANT_API_KEY, driver)

# get the SEASON of the last game in the database
# this will used when constructing rows for prediction
SEASON = df_new['SEASON'].max()

df_new




,GAME_DATE_EST,HOME_TEAM_WINS,PTS_home,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,HOME_TEAM_ID,GAME_ID,PTS_away,FG_PCT_away,FG3_PCT_away,FT_PCT_away,REB_away,AST_away,VISITOR_TEAM_ID,SEASON
0,2023-04-09,0,81,36.5,27.3,76.9,59,20,1610612765,22201223,103,42.1,20.0,77.3,48,21,1610612741,2022
1,2023-04-09,0,108,44.4,14.3,86.2,49,18,1610612740,22201225,113,47.6,36.1,71.0,42,25,1610612750,2022
2,2023-04-09,1,157,60.4,55.1,87.5,58,47,1610612744,22201230,101,40.0,25.7,66.7,30,25,1610612757,2022
3,2023-04-09,1,119,49.0,21.7,66.7,53,22,1610612746,22201229,114,42.4,37.8,72.7,47,29,1610612756,2022
4,2023-04-09,0,117,50.5,28.6,83.3,45,31,1610612762,22201228,128,48.5,40.0,76.2,49,28,1610612747,2022
5,2023-04-09,0,95,43.5,25.6,66.7,34,24,1610612758,22201227,109,50.0,25.0,72.0,51,25,1610612743,2022
6,2023-04-09,0,100,43.5,18.9,72.2,43,25,1610612763,22201226,115,50.0,33.3,81.3,49,30,1610612760,2022
7,2023-04-09,1,138,49.0,39.5,70.0,69,31,1610612759,22201224,117,42.6,32.1,50.0,46,28,1610612742,2022
8,2023-04-09,1,141,51.7,55.9,78.9,44,33,1610612754,22201220,136,48.1,37.5,77.3,53,29,1610612752,2022
9,2023-04-09,0,110,42.2,36.2,70.8,43,29,1610612753,22201219,123,54.2,40.9,75.0,44,30,1610612748,2022


**Retrieve todays games**

In [5]:
#retrieve list of teams playing today

# get today's games on NBA schedule
matchups, game_ids = get_todays_matchups(SCRAPINGANT_API_KEY, driver)

if matchups is None:
    print('No games today')
else:
    print(matchups)
    print(game_ids)


Tue
Wed
Fri
Sat
Sun
Sun
No games today


**Close Webdriver**

In [6]:
if WEBSCRAPER == 'SELENIUM':
    driver.close() 

**Create Rows for Today's Games with Empty Stats**

In [7]:
# append today's matchups to the new games dataframe

if matchups is None:
    print('No games today')
else:

    df_today = df_new.drop(df_new.index) #empty copy of df_new with same columns
    for i, matchup in enumerate(matchups):
        game_details = {'HOME_TEAM_ID': matchup[1], 
                        'VISITOR_TEAM_ID': matchup[0], 
                        'GAME_DATE_EST': datetime.now(timezone('EST')).strftime("%Y-%m-%d"), 
                        'GAME_ID': int(game_ids[i]),                       
                        'SEASON': SEASON,
                        } 
        game_details_df = pd.DataFrame(game_details, index=[i])
        # append to new games dataframe
        df_today = pd.concat([df_today, game_details_df], ignore_index = True)

    #blank rows will be filled with 0 to prevent issues with feature engineering
    df_today = df_today.fillna(0) 

    df_today


No games today


**Access Feature Store**

In [8]:
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)

# HOPSWORKS can be kinda buggy and has been throwing a lot of errors recently or even just failing to return data
# so I'm adding a try/except block to retry the query if it fails
tries = 5

for i in range(tries):
    
    try:
        fs = project.get_feature_store()
    except KeyError as e:
        if i < tries - 1: # i is zero indexed
            time.sleep(30)
            continue
        else:
            raise ValueError('HOPSWORKS failed to connect')
    break



Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/3350


Connected. Call `.close()` to terminate connection gracefully.


**Access Feature Group**

In [9]:
# HOPSWORKS can be kinda buggy and has been throwing a lot of errors recently or even just failing to return data
# so I'm adding a try/except block to retry the query if it fails
tries = 5

for i in range(tries):
    
    try:
        rolling_stats_fg = fs.get_feature_group(
        name="rolling_stats",
        version=2,
        )
    except KeyError as e:
        if i < tries - 1: # i is zero indexed
            time.sleep(30)
            continue
        else:
            raise ValueError('HOPSWORKS failed to connect')
    break



**Query Old Data Needed for Feature Engineering of New Data**

To generate features like rolling averages for the new games, older data from previous games is needed since some of the rolling averages might extend back 15 or 20 games or so.

In [10]:
BASE_FEATURES = ['game_date_est',
 'game_id',
 'home_team_id',
 'visitor_team_id',
 'season',
 'pts_home',
 'fg_pct_home',
 'ft_pct_home',
 'fg3_pct_home',
 'ast_home',
 'reb_home',
 'pts_away',
 'fg_pct_away',
 'ft_pct_away',
 'fg3_pct_away',
 'ast_away',
 'reb_away',
 'home_team_wins',
]

ds_query = rolling_stats_fg.select(BASE_FEATURES)

# HOPSWORKS can be kinda buggy and has been throwing a lot of errors recently or even just failing to return data
# so I'm adding a try/except block to retry the query if it fails
tries = 5

for i in range(tries):
    for j in range(tries):
        try:
            df_old = ds_query.read()
        except KeyError as e:
            if j < tries - 1: 
                time.sleep(10)
                continue
            else:
                raise ValueError('HOPSWORKS failed to connect')
        break

    if df_old.empty:
        if i < tries - 1: 
            time.sleep(10)
        else:
            raise ValueError('HOPSWORKS failed to return data')
    else:
        break



df_old


2023-04-10 08:02:55,500 INFO: USE `nba_predictor_featurestore`
2023-04-10 08:02:55,963 INFO: SELECT `fg0`.`game_date_est` `game_date_est`, `fg0`.`game_id` `game_id`, `fg0`.`home_team_id` `home_team_id`, `fg0`.`visitor_team_id` `visitor_team_id`, `fg0`.`season` `season`, `fg0`.`pts_home` `pts_home`, `fg0`.`fg_pct_home` `fg_pct_home`, `fg0`.`ft_pct_home` `ft_pct_home`, `fg0`.`fg3_pct_home` `fg3_pct_home`, `fg0`.`ast_home` `ast_home`, `fg0`.`reb_home` `reb_home`, `fg0`.`pts_away` `pts_away`, `fg0`.`fg_pct_away` `fg_pct_away`, `fg0`.`ft_pct_away` `ft_pct_away`, `fg0`.`fg3_pct_away` `fg3_pct_away`, `fg0`.`ast_away` `ast_away`, `fg0`.`reb_away` `reb_away`, `fg0`.`home_team_wins` `home_team_wins`
FROM `nba_predictor_featurestore`.`rolling_stats_2` `fg0`


,game_date_est,game_id,home_team_id,visitor_team_id,season,pts_home,fg_pct_home,ft_pct_home,fg3_pct_home,ast_home,reb_home,pts_away,fg_pct_away,ft_pct_away,fg3_pct_away,ast_away,reb_away,home_team_wins
0,2018-01-05,21700570,1610612738,1610612750,2017,91,0.404053,0.691895,0.166992,24,56,84,0.367920,0.750000,0.303955,15,43,1
1,2006-12-11,20600301,1610612753,1610612756,2006,89,0.451904,0.523926,0.166992,18,42,103,0.514160,0.852051,0.250000,25,36,0
2,2007-11-10,20700084,1610612758,1610612750,2007,100,0.405029,0.850098,0.399902,12,38,93,0.438965,0.750000,0.213989,15,43,1
3,2005-01-11,20400504,1610612760,1610612746,2004,104,0.437988,0.889160,0.500000,24,40,99,0.438965,0.758789,0.333008,23,47,1
4,2016-10-29,21600028,1610612752,1610612763,2016,111,0.525879,0.641113,0.285889,24,41,104,0.429932,0.808105,0.360107,19,40,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23992,2017-12-10,21700387,1610612754,1610612743,2017,126,0.526855,0.817871,0.399902,24,40,116,0.452881,0.812988,0.405029,24,47,1
23993,2015-04-10,21401175,1610612753,1610612761,2014,99,0.460938,1.000000,0.350098,21,46,101,0.415039,0.933105,0.300049,22,46,0
23994,2005-01-15,20400537,1610612745,1610612759,2004,73,0.333008,0.730957,0.399902,15,49,67,0.353027,0.629883,0.125000,10,39,1
23995,2012-03-07,21100575,1610612749,1610612741,2011,104,0.477051,0.789062,0.333008,29,35,106,0.518066,0.881836,0.312988,26,43,0


**Convert Feature Names back to original mixed case**

In [11]:
#hopsworks converts all feature names to lowercase, and for code reuse, we need to convert them back
df_old = convert_feature_names(df_old)
df_old
df_old[df_old['PTS_home'] == 0]

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
1353,2023-04-09,22201217,1610612751,1610612755,2022,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0
2312,2023-04-09,22201226,1610612760,1610612763,2022,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0
3134,2023-04-09,22201218,1610612739,1610612766,2022,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0
3632,2023-04-09,22201219,1610612748,1610612753,2022,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0
5143,2023-04-09,22201220,1610612752,1610612754,2022,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0
6589,2023-04-09,22201227,1610612743,1610612758,2022,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0
9465,2023-04-09,22201221,1610612761,1610612749,2022,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0
11068,2023-04-09,22201230,1610612757,1610612744,2022,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0
12934,2023-04-09,22201223,1610612741,1610612765,2022,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0
14257,2023-04-09,22201225,1610612750,1610612740,2022,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0


**Update Yesterday's Matchup Predictions with New Final Results**

In [12]:
# filter out games that are pending final results
# (these were the rows used for prediction yesterday)
# and then update these with the new results


# one approach is to simply drop the rows that were used for prediction yesterday
# which are games that have 0 points for home team
# and then append the new rows to the dataframe
df_old = df_old[df_old['PTS_home'] != 0]
df_old = pd.concat([df_old, df_new], ignore_index = True)

df_old

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2018-01-05,21700570,1610612738,1610612750,2017,91,0.404053,0.691895,0.166992,24,56,84,0.367920,0.750000,0.303955,15,43,1
1,2006-12-11,20600301,1610612753,1610612756,2006,89,0.451904,0.523926,0.166992,18,42,103,0.514160,0.852051,0.250000,25,36,0
2,2007-11-10,20700084,1610612758,1610612750,2007,100,0.405029,0.850098,0.399902,12,38,93,0.438965,0.750000,0.213989,15,43,1
3,2005-01-11,20400504,1610612760,1610612746,2004,104,0.437988,0.889160,0.500000,24,40,99,0.438965,0.758789,0.333008,23,47,1
4,2016-10-29,21600028,1610612752,1610612763,2016,111,0.525879,0.641113,0.285889,24,41,104,0.429932,0.808105,0.360107,19,40,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,2023-04-09,22201216,1610612737,1610612738,2022,114,45.400000,78.900000,28.200000,27,50,120,47.200000,78.600000,46.300000,33,47,0
23996,2023-04-09,22201221,1610612749,1610612761,2022,105,46.300000,81.000000,35.300000,28,38,121,50.500000,87.500000,33.300000,29,45,0
23997,2023-04-08,22201214,1610612750,1610612759,2022,151,55.400000,71.400000,55.800000,42,45,131,52.600000,75.000000,44.800000,34,45,1
23998,2023-04-08,22201215,1610612757,1610612746,2022,125,52.800000,73.900000,43.800000,32,34,136,52.700000,86.700000,34.300000,30,51,0


**Add Today's Matchups for Feature Engineering**

In [13]:
if matchups is None:
    print('No games today')
    df_combined = df_old
else:
    df_combined = pd.concat([df_old, df_today], ignore_index = True)
    df_combined

No games today


**Data Processing**

In [14]:
df_combined = process_games(df_combined) 
df_combined = add_TARGET(df_combined)
df_combined

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,PLAYOFF,TARGET
0,2018-01-05,21700570,1610612738,1610612750,2017,91,0.404053,0.691895,0.166992,24,56,84,0.367920,0.750000,0.303955,15,43,1,0,1
1,2006-12-11,20600301,1610612753,1610612756,2006,89,0.451904,0.523926,0.166992,18,42,103,0.514160,0.852051,0.250000,25,36,0,0,0
2,2007-11-10,20700084,1610612758,1610612750,2007,100,0.405029,0.850098,0.399902,12,38,93,0.438965,0.750000,0.213989,15,43,1,0,1
3,2005-01-11,20400504,1610612760,1610612746,2004,104,0.437988,0.889160,0.500000,24,40,99,0.438965,0.758789,0.333008,23,47,1,0,1
4,2016-10-29,21600028,1610612752,1610612763,2016,111,0.525879,0.641113,0.285889,24,41,104,0.429932,0.808105,0.360107,19,40,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23992,2023-04-09,22201218,1610612766,1610612739,2022,106,45.700000,86.700000,18.800000,23,49,95,39.300000,93.800000,27.000000,27,44,1,0,1
23993,2023-04-09,22201222,1610612745,1610612764,2022,114,49.400000,71.400000,28.600000,24,51,109,42.700000,61.900000,31.100000,28,49,1,0,1
23994,2023-04-09,22201217,1610612755,1610612751,2022,134,52.500000,73.700000,43.800000,31,50,105,42.200000,82.100000,37.500000,22,42,1,0,1
23995,2023-04-09,22201216,1610612737,1610612738,2022,114,45.400000,78.900000,28.200000,27,50,120,47.200000,78.600000,46.300000,33,47,0,0,0


**Feature Engineering**

In [15]:
# Feature engineering to add: 
    # rolling averages of key stats, 
    # win/lose streaks, 
    # home/away streaks, 
    # specific matchup (team X vs team Y) rolling averages and streaks

df_combined = process_features(df_combined)



#fix type conversion issues with hopsworks
df_combined['TARGET'] = df_combined['TARGET'].astype('int16')
df_combined['HOME_TEAM_WINS'] = df_combined['HOME_TEAM_WINS'].astype('int16')

df_combined


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


['HOME_PTS_home_AVG_LAST_3_HOME', 'HOME_PTS_home_AVG_LAST_7_HOME', 'HOME_PTS_home_AVG_LAST_10_HOME', 'HOME_FG_PCT_home_AVG_LAST_3_HOME', 'HOME_FG_PCT_home_AVG_LAST_7_HOME', 'HOME_FG_PCT_home_AVG_LAST_10_HOME', 'HOME_FT_PCT_home_AVG_LAST_3_HOME', 'HOME_FT_PCT_home_AVG_LAST_7_HOME', 'HOME_FT_PCT_home_AVG_LAST_10_HOME', 'HOME_FG3_PCT_home_AVG_LAST_3_HOME', 'HOME_FG3_PCT_home_AVG_LAST_7_HOME', 'HOME_FG3_PCT_home_AVG_LAST_10_HOME', 'HOME_AST_home_AVG_LAST_3_HOME', 'HOME_AST_home_AVG_LAST_7_HOME', 'HOME_AST_home_AVG_LAST_10_HOME', 'HOME_REB_home_AVG_LAST_3_HOME', 'HOME_REB_home_AVG_LAST_7_HOME', 'HOME_REB_home_AVG_LAST_10_HOME', 'HOME_TEAM_ID', 'GAME_DATE_EST']


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


['VISITOR_TEAM_WINS_AVG_LAST_3_VISITOR', 'VISITOR_TEAM_WINS_AVG_LAST_7_VISITOR', 'VISITOR_TEAM_WINS_AVG_LAST_10_VISITOR', 'VISITOR_PTS_away_AVG_LAST_3_VISITOR', 'VISITOR_PTS_away_AVG_LAST_7_VISITOR', 'VISITOR_PTS_away_AVG_LAST_10_VISITOR', 'VISITOR_FG_PCT_away_AVG_LAST_3_VISITOR', 'VISITOR_FG_PCT_away_AVG_LAST_7_VISITOR', 'VISITOR_FG_PCT_away_AVG_LAST_10_VISITOR', 'VISITOR_FT_PCT_away_AVG_LAST_3_VISITOR', 'VISITOR_FT_PCT_away_AVG_LAST_7_VISITOR', 'VISITOR_FT_PCT_away_AVG_LAST_10_VISITOR', 'VISITOR_FG3_PCT_away_AVG_LAST_3_VISITOR', 'VISITOR_FG3_PCT_away_AVG_LAST_7_VISITOR', 'VISITOR_FG3_PCT_away_AVG_LAST_10_VISITOR', 'VISITOR_AST_away_AVG_LAST_3_VISITOR', 'VISITOR_AST_away_AVG_LAST_7_VISITOR', 'VISITOR_AST_away_AVG_LAST_10_VISITOR', 'VISITOR_REB_away_AVG_LAST_3_VISITOR', 'VISITOR_REB_away_AVG_LAST_7_VISITOR', 'VISITOR_REB_away_AVG_LAST_10_VISITOR', 'VISITOR_TEAM_ID', 'GAME_DATE_EST']


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


['PTS_AVG_LAST_3_ALL', 'PTS_AVG_LAST_7_ALL', 'PTS_AVG_LAST_10_ALL', 'PTS_AVG_LAST_15_ALL', 'FG_PCT_AVG_LAST_3_ALL', 'FG_PCT_AVG_LAST_7_ALL', 'FG_PCT_AVG_LAST_10_ALL', 'FG_PCT_AVG_LAST_15_ALL', 'FT_PCT_AVG_LAST_3_ALL', 'FT_PCT_AVG_LAST_7_ALL', 'FT_PCT_AVG_LAST_10_ALL', 'FT_PCT_AVG_LAST_15_ALL', 'FG3_PCT_AVG_LAST_3_ALL', 'FG3_PCT_AVG_LAST_7_ALL', 'FG3_PCT_AVG_LAST_10_ALL', 'FG3_PCT_AVG_LAST_15_ALL', 'AST_AVG_LAST_3_ALL', 'AST_AVG_LAST_7_ALL', 'AST_AVG_LAST_10_ALL', 'AST_AVG_LAST_15_ALL', 'REB_AVG_LAST_3_ALL', 'REB_AVG_LAST_7_ALL', 'REB_AVG_LAST_10_ALL', 'REB_AVG_LAST_15_ALL', 'TEAM1', 'GAME_DATE_EST']


,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,...,FG3_PCT_AVG_LAST_10_ALL_x_minus_y,FG3_PCT_AVG_LAST_15_ALL_x_minus_y,AST_AVG_LAST_3_ALL_x_minus_y,AST_AVG_LAST_7_ALL_x_minus_y,AST_AVG_LAST_10_ALL_x_minus_y,AST_AVG_LAST_15_ALL_x_minus_y,REB_AVG_LAST_3_ALL_x_minus_y,REB_AVG_LAST_7_ALL_x_minus_y,REB_AVG_LAST_10_ALL_x_minus_y,REB_AVG_LAST_15_ALL_x_minus_y
0,2003-10-28,20300003,1610612747,1610612742,2003,109,0.505859,0.600098,0.350098,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003-10-28,20300001,1610612755,1610612748,2003,89,0.439941,0.533203,0.350098,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2003-10-28,20300002,1610612759,1610612756,2003,83,0.425049,0.769043,0.099976,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2003-10-29,20300006,1610612740,1610612737,2003,88,0.323975,0.700195,0.160034,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2003-10-29,20300010,1610612741,1610612764,2003,74,0.316895,0.612793,0.230957,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23992,2023-04-09,22201218,1610612766,1610612739,2022,106,45.687500,86.687500,18.796875,23,...,-3.798438,-2.192708,-0.666667,3.000000,1.7,1.333333,1.666667,5.714286,4.2,4.000000
23993,2023-04-09,22201227,1610612758,1610612743,2022,95,43.500000,66.687500,25.593750,24,...,0.201563,3.008333,2.666667,2.571429,1.5,0.600000,4.666667,2.714286,0.7,3.133333
23994,2023-04-09,22201224,1610612759,1610612742,2022,138,49.000000,70.000000,39.500000,31,...,-4.689062,-1.814583,5.666667,3.571429,3.4,3.866667,0.333333,5.000000,4.6,5.333333
23995,2023-04-09,22201230,1610612744,1610612757,2022,157,60.406250,87.500000,55.093750,47,...,4.107813,4.817708,-0.666667,4.000000,5.0,5.066667,13.000000,8.857143,8.5,7.733333


**Insert New Data into Feature Group**

In [16]:
# HOPSWORKS can be kinda buggy and has been throwing a lot of errors recently or even just failing to return data
# so I'm adding a try/except block to retry the query if it fails
tries = 5

for i in range(tries):
    
    try:
        rolling_stats_fg.insert(df_combined, overwrite = True, write_options={"wait_for_job" : False})
    except KeyError as e:
        if i < tries - 1: 
            time.sleep(30)
            continue
        else:
            raise ValueError('HOPSWORKS failed to connect')
    break



Uploading Dataframe: 0.00% |          | Rows 0/23997 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/3350/jobs/named/rolling_stats_2_offline_fg_backfill/executions


In [17]:
df_combined

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,...,FG3_PCT_AVG_LAST_10_ALL_x_minus_y,FG3_PCT_AVG_LAST_15_ALL_x_minus_y,AST_AVG_LAST_3_ALL_x_minus_y,AST_AVG_LAST_7_ALL_x_minus_y,AST_AVG_LAST_10_ALL_x_minus_y,AST_AVG_LAST_15_ALL_x_minus_y,REB_AVG_LAST_3_ALL_x_minus_y,REB_AVG_LAST_7_ALL_x_minus_y,REB_AVG_LAST_10_ALL_x_minus_y,REB_AVG_LAST_15_ALL_x_minus_y
0,2003-10-28,20300003,1610612747,1610612742,2003,109,0.505859,0.600098,0.350098,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003-10-28,20300001,1610612755,1610612748,2003,89,0.439941,0.533203,0.350098,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2003-10-28,20300002,1610612759,1610612756,2003,83,0.425049,0.769043,0.099976,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2003-10-29,20300006,1610612740,1610612737,2003,88,0.323975,0.700195,0.160034,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2003-10-29,20300010,1610612741,1610612764,2003,74,0.316895,0.612793,0.230957,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23992,2023-04-09,22201218,1610612766,1610612739,2022,106,45.687500,86.687500,18.796875,23,...,-3.798438,-2.192708,-0.666667,3.000000,1.7,1.333333,1.666667,5.714286,4.2,4.000000
23993,2023-04-09,22201227,1610612758,1610612743,2022,95,43.500000,66.687500,25.593750,24,...,0.201563,3.008333,2.666667,2.571429,1.5,0.600000,4.666667,2.714286,0.7,3.133333
23994,2023-04-09,22201224,1610612759,1610612742,2022,138,49.000000,70.000000,39.500000,31,...,-4.689062,-1.814583,5.666667,3.571429,3.4,3.866667,0.333333,5.000000,4.6,5.333333
23995,2023-04-09,22201230,1610612744,1610612757,2022,157,60.406250,87.500000,55.093750,47,...,4.107813,4.817708,-0.666667,4.000000,5.0,5.066667,13.000000,8.857143,8.5,7.733333


In [18]:
# check to make sure there are no duplicate games were inadvertently added
df_combined[df_combined.duplicated(subset=['GAME_ID'], keep=False)]

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,...,FG3_PCT_AVG_LAST_10_ALL_x_minus_y,FG3_PCT_AVG_LAST_15_ALL_x_minus_y,AST_AVG_LAST_3_ALL_x_minus_y,AST_AVG_LAST_7_ALL_x_minus_y,AST_AVG_LAST_10_ALL_x_minus_y,AST_AVG_LAST_15_ALL_x_minus_y,REB_AVG_LAST_3_ALL_x_minus_y,REB_AVG_LAST_7_ALL_x_minus_y,REB_AVG_LAST_10_ALL_x_minus_y,REB_AVG_LAST_15_ALL_x_minus_y
